## CDCS_MP

This notebook obtains and performs some image analysis on optical images taken under bright field (BF) imaging conditions from the ambench.nist.gov database.

In [1]:
import pandas as py             # a standard Python data processing library
import lxml.etree as et
from cdcs import CDCS
from urllib import request
from skimage import io

### 1. Importing records from CDCS
 - uses the CDCS REST API client to search for datasets related to the melt pool (MP) challenge within the ambench.nist.gov   instance
 - searches the returned datasets for tiff images containing the phrase "BF" and collects information about the corresponding track and case for each image
 - stores image names, image data, track, and case in a pandas dataframe

create a CDCS query instance and input a keyword

In [ ]:
keyword='MP'
curator = CDCS('https://ambench.nist.gov/', username='') #accessing anonymously
df=curator.query(template='AM-Bench-2018',keyword=keyword) #searching for results with the phrase'MP'
df #display the dataframe

#### xml_url_find:
Parameters:
- XML file for one of the ambench.nist.gov datasets in string format
- search phrase contained in the file name
- file type

Returns:
- a list containing (0) name of a file, (1) the download url for the file, (2) laser track number, (3) case 

In [ ]:
def xml_url_find(xml,searchphrase,mtype):
    root=et.fromstring(xml)
    caseid=root.find('.//TraceID')[0].tag[5]
    track=root.find('.//TrackNumber')
    for element in root.iter('downloadURL'):
        u=request.urlopen(element.text)
        if searchphrase in u.info().get_filename() and mtype in u.info().get_content_type(): #looking for searchphrase in
            #filename and mtype in file type
            name=u.info().get_filename()
            url=element.text
            return [name,url,track.text,caseid]

In [ ]:
searchphrase='BF' #returns a list of scikit images
filename=[]
result=[]
trace=[]
case=[]
for index in df.index:
    res=xml_url_find(df.iloc[index]['xml_content'],'BF','image/tiff') #giving xml_url_find each xml string and searching for
    #tiff files with 'BF' in their name 
    name=res[0]
    url=res[1]
    track=res[2]
    c=res[3]
    if url==None:
        continue
    else:
        filename.append(name)
        case.append(c)
        result.append(io.imread(url)) #reading in the urls using scikit
        trace.append(track)
dfres=py.DataFrame({'name':filename,'image':result,'track':trace,'case':case}) #stores images and image names in a dataframe
dfres

### 2. Image visualization and analysis
- using scikit-image to perform image analysis and matplotlib to display images
- uses the felzenszwalb image segmentation algorithm to help obtain widths and depths of the melt pools
- displays plots of width and depth
- stores measurements in a csv file

In [ ]:
import skimage
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#### draw_box_opt
This method (1) crops the melt pool images (to reduce runtime), (2) applies the felzenszwalb segmentation algorithm to the image, (3) removes smaller regions (eliminate some background noise), (4) acquires a table of center coordinates for the remaining regions, (5) measures the distances from centers to the center of the image, (6) finds the index number of the closest region, (7) returns information about the closes region

Parameters:
- image to be analyzed

Returns:
- list of RegionProperties related to the melt pool region

In [ ]:
from skimage import filters
from skimage import segmentation
from skimage import morphology
from skimage import measure
import math
def draw_box_opt(image):
    cropim=image[900:1800,800:3400] #cropping so that there's less pixels to cover - incredibly slow if left at original size
    segments=segmentation.felzenszwalb(cropim,scale=270,sigma=0.8,min_size=1000)
    isolateim=morphology.remove_small_objects(segments,100000)
    center=measure.regionprops_table(isolateim,properties=['centroid']) #table of information about the center points of regions
    distances=[]
    for n in range(len(center['centroid-0'])):
        distances.append(math.dist([center['centroid-0'][n],center['centroid-1'][n]],[450,1300]))
    index=distances.index(min(distances)) #Looking for the region closest to the center of the image - should be the melt pool
    object_features=measure.regionprops(isolateim)
    return object_features[index] 

In [ ]:
depths=[]
widths=[]
bboxes=[]
for im in dfres['image']:
    objf=draw_box_opt(im)
    bboxes.append(objf.bbox)
    minr,minc,maxr,maxc=objf.bbox #using the bbox drawn around the melt pool to find width and depth
    widths.append((maxc-minc)*(0.062)) #using the conversion given in the CDCS comment above the images to convert to mm (1 pixel=0.062 microns)
    depths.append((maxr-minr)*(0.062))
dat={'width':widths,'depth':depths,'track':trace,'case':case}

In [ ]:
dfmp=py.DataFrame(dat)

In [ ]:
dfmp

Bar plot of melt pool widths

In [ ]:
dfmp.plot(y='width',x='track',kind='bar')

Bar plot of melt pool depths

In [ ]:
dfmp.plot(y='depth',x='track',kind='bar')

Example of what the image looks like with the bbox marked

In [ ]:
fig, ax = plt.subplots()
minr,minc,maxr,maxc=bboxes[4]
ax.imshow(result[4][900:1800,800:3400])
bx = (minc, maxc, maxc, minc, minc)
by = (minr, minr, maxr, maxr, minr)
ax.plot(bx, by, '-b', linewidth=2.5)
plt.show()

Putting the calculated measurements into a csv file and saving it

In [ ]:
dfmp.to_csv('MP_measurements.csv',index=False)